In the NASA dataset, the first row of each cycle has both voltage and time values as zero, so we need to delete it. Also, when the voltage data drops to zero, the time continues to count, but we need to reset it. This file addresses the above issues, and the processed data is used for feature extraction.

In [1]:
import pandas as pd

# Load the Excel file
file_path = "./datacsv/battery_discharge_data_all.xlsx"  # Replace with your path
xls = pd.ExcelFile(file_path)

# Get all sheet names
sheet_names = xls.sheet_names

RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

In [2]:
# Define the processing function
def process_discharge_sheet(df):
    # Clean up column names
    df.columns = df.columns.str.strip()
    if 'cycle' not in df.columns or 'voltage' not in df.columns:
        return df  # Skip sheets without necessary columns
    
    processed = []
    for cycle_id, group in df.groupby('cycle'):
        group = group.copy()
        
        # Delete the first row of each cycle
        group = group.iloc[1:]

        # If there is a Voltage = 0, truncate the data after that row (keep that row)
        zero_voltage_idx = group[group['voltage'] == 0].index
        if not zero_voltage_idx.empty:
            first_zero_idx = zero_voltage_idx[0]
            group = group.loc[:first_zero_idx]

        processed.append(group)

    return pd.concat(processed, ignore_index=True)

In [3]:
# Process all _discharge sheets
processed_sheets = {}
for sheet in sheet_names:
    print(sheet)
    if sheet.endswith('_discharge'):
        df = xls.parse(sheet)
        processed_df = process_discharge_sheet(df)
        processed_sheets[sheet] = processed_df

# Save as a new Excel file
output_path = "./datacsv/processed_battery_discharge_data.xlsx"
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    for sheet_name, df in processed_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Processing complete, saved as: {output_path}")

B0036_discharge
B0018_discharge
B0007_discharge
B0039_discharge
B0045_discharge
B0041_discharge
B0054_discharge
B0050_discharge
B0056_discharge
B0055_discharge
B0049_discharge
B0028_discharge
B0030_discharge
B0042_discharge
B0034_discharge
B0040_discharge
B0005_discharge
B0048_discharge
B0043_discharge
B0052_discharge
B0031_discharge
B0051_discharge
B0038_discharge
B0032_discharge
B0029_discharge
B0046_discharge
B0006_discharge
B0027_discharge
B0033_discharge
B0053_discharge
B0026_discharge
B0025_discharge
B0047_discharge
B0044_discharge
index
Processing complete, saved as: ./datacsv/processed_battery_discharge_data.xlsx


In [4]:
import pandas as pd

excel_path = './datacsv/processed_battery_discharge_data.xlsx'
xls = pd.read_excel(excel_path, sheet_name=None)

filtered_sheets = {}

for sheet_name, df in xls.items():
    if 'time' in df.columns:
        df_filtered = df[df['time'] <= 2520]
        filtered_sheets[sheet_name] = df_filtered
    else:
        filtered_sheets[sheet_name] = df

output_path = './datacsv/processed_battery_discharge_data_filtered.xlsx'
with pd.ExcelWriter(output_path) as writer:
    for sheet_name, df in filtered_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Processing completed.")

Processing completed.
